## European and American

In [36]:
import math
r=0.05
T=2
t=T/2
u=1.2
d=0.8

q=(math.exp(r*t)-d)/(u-d)
print('q is',q)

q is 0.6281777409400603


## Store common attributes of a stock option

In [37]:
## store common attributes of a stock option
class StockOption(object):
    def __init__(self,S0,K,r=0.05,T=1,N=2,pu=0,pd=0,div=0,sigma=0,is_put=False,is_am=False):
        self.S0=S0
        self.K=K
        self.r=r
        self.T=T
        self.N=max(1,N)
        self.STs=[] #stock price tree
        
        self.pu,self.pd=pu,pd
        self.div=div
        self.sigma=sigma
        self.is_call=not is_put
        self.is_european=not is_am
        
    @property
    def dt(self):
        return self.T/float(self.N) # single time strp, in years
    @property
    def df(self):
        return math.exp(-(self.r-self.div)*self.dt) # discount factor
        

## EU Binomal Tree

In [38]:
import math
import numpy as np
from decimal import Decimal

class BinominalEuropeanOption(StockOption):
    def setup_parameters(self):
        # required calculations for the model
        self.M=self.N+1 # number of terminal nodes for tree
        self.u=1+self.pu
        self.d=1-self.pd
        self.qu=(math.exp(self.r-self.div)*self.dt-self.d)/(self.u-self.d)
        self.qd=1-self.qu
    def init_stock_price_tree(self):
        self.STs=np.zeros(self.M)
        for i in range(self.M):
            self.STs[i]=self.S0*(self.u**(self.N-i))*(self.d**i)
    def init_payoffs_tree(self):
        if self.is_call:
            return np.maximum(0,self.STs-self.K)
        else:
            return np.maximum(0,self.K-self.STs)
    def traverse_tree(self,payoffs):
        for i in range(self.N):
            payoffs=(payoffs[:-1]*self.qu+payoffs[1:]*self.qd)*self.df # final stage
        return payoffs
    def begin_tree_traversal(self):
        payoffs=self.init_payoffs_tree()
        return self.traverse_tree(payoffs)
    def price(self):
        self.setup_parameters()
        self.init_stock_price_tree()
        payoffs=self.begin_tree_traversal()
        return payoffs[0]

In [39]:
eu_option=BinominalEuropeanOption(50,52,r=0.05,T=2,N=2,pu=0.2,pd=0.2,is_put=True)
print('EO put option price is:',eu_option.price())

EO put option price is: 4.1926542806038585


## American Binomal Tree

In [40]:
# M parameter is not needed

In [41]:
class BinomialTreeOption(StockOption): # create a new class name
    def setup_parameters(self):
        # required calculations for the model
        self.u=1+self.pu
        self.d=1-self.pd
        self.qu=(math.exp(self.r-self.div)*self.dt-self.d)/(self.u-self.d)
        self.qd=1-self.qu
    def init_stock_price_tree(self):
        # create a 2D tree at T=0 to store returns of each step
        self.STs=[np.array([self.S0])]
        for i in range(self.N):
            prev_branches=self.STs[-1]
            st=np.concatenate((prev_branches*self.u,[prev_branches[-1]*self.d]))
            self.STs.append(st)
    def init_payoffs_tree(self):
        # calculate intrinsic value in each stage, add[self.N]
        if self.is_call:
            return np.maximum(0,self.STs[self.N]-self.K)
        else:
            return np.maximum(0,self.K-self.STs[self.N])
    # create a new function to check early excerise
    def check_early_excerise(self,payoffs,node):
        if self.is_call:
            return np.maximum(payoffs,self.STs[node]-self.K)
        else:
            return np.maximum(payoffs,self.K-self.STs[node])
    def traverse_tree(self,payoffs): # should include function: check_early_excerise()
        for i in reversed(range(self.N)): # reversed: start from final stage
            payoffs=(payoffs[:-1]*self.qu+payoffs[1:]*self.qd)*self.df # of not excerising
            if not self.is_european:
                payoffs=self.check_early_excerise(payoffs,i) # for excerising, American
        return payoffs
    def begin_tree_traversal(self):
        payoffs=self.init_payoffs_tree()
        return self.traverse_tree(payoffs)
    def price(self):
        self.setup_parameters()
        self.init_stock_price_tree()
        payoffs=self.begin_tree_traversal()
        return payoffs[0]

In [42]:
am_option=BinomialTreeOption(50,52,r=0.05,T=2,N=2,pu=0.2,pd=0.2,is_put=True,is_am=True)
print('American put option price is:',am_option.price())

American put option price is: 5.089632474198373


## CRR Binomial Tree

In [43]:
# change the u and d, the rest is the same with binomial tree model
class BinomialCRROption(BinomialTreeOption):
    def setup_parameters(self):
        self.u=math.exp(self.sigma*math.sqrt(self.dt))
        self.d=1/self.u
        self.qu=(math.exp((self.r-self.div)*self.dt)-self.d)/(self.u-self.d)
        self.qd=1-self.qu

In [44]:
eu_option=BinomialCRROption(50,52,r=0.05,T=2,N=2,sigma=0.3,is_put=True)
print('EU put:',eu_option.price())

EU put: 6.245708445206436


In [45]:
am_option=BinomialCRROption(50,52,r=0.05,T=2,N=2,sigma=0.3,is_put=True, is_am=True)
print('American put:',am_option.price())

American put: 7.428401902704834


## LR Binomial Tree

In [46]:
class BinomialLROption(BinomialTreeOption):
    def setup_parameters(self):
        odd_N=self.N if (self.N%2==0) else (self.N+1)
        d1=(math.log(self.S0/self.K)+((self.r-self.div)+(self.sigma**2)/2)*self.T)/(self.sigma*math.sqrt(self.T))
        d2=(math.log(self.S0/self.K)-((self.r-self.div)+(self.sigma**2)/2)*self.T)/(self.sigma*math.sqrt(self.T))
        pbar=self.pp_2_inversion(d1,odd_N)
        self.p=self.pp_2_inversion(d2,odd_N)
        self.u=1/self.df*pbar/self.p
        self.d=(1/self.df)*(1-pbar)/(1-self.p)
        self.qu=self.p
        self.qd=1-self.p
    def pp_2_inversion(self,z,n):
        return 0.5+math.copysign(1,z)*math.sqrt(0.25-0.25*math.exp(-((z/(n+(1/3)+0.1/(n+1)))**2)*(n+(1/6))))

In [47]:
eu_option=BinomialLROption(50,52,r=0.05,T=2,N=4,sigma=0.3,is_put=True)
print('EU put:',eu_option.price())

EU put: 14.399949438594852


In [48]:
am_option=BinomialLROption(50,52,r=0.05,T=2,N=4,sigma=0.3,is_put=True, is_am=True)
print('American put:',am_option.price())

American put: 14.971574642640627


## Greeks with LR Binomial Tree

In [62]:
class BinomialLRWithGreeks(BinomialLROption):
    def new_stock_price_tree(self):
        self.STs=[np.array([self.S0*self.u/self.d,self.S0,self.S0*self.d/self.u])]
        for i in range(self.N):
            prev_branches=self.STs[-1]
            st=np.concatenate((prev_branches*self.u,[prev_branches[-1]*self.d]))
            self.STs.append(st)
    def price(self):
        self.setup_parameters()
        self.new_stock_price_tree()
        payoffs=self.begin_tree_traversal()
        option_value=payoffs[len(payoffs)//2]
        payoff_up=payoffs[0]
        payoff_down=payoffs[-1]
        S_up=self.STs[0][0]
        S_down=self.STs[0][-1]
        dS_up=S_up-self.S0
        dS_down=self.S0-S_down
        # delta
        ds=S_up-S_down
        dV=payoff_up-payoff_down
        delta=dV/ds
        # gamma
        gamma=((payoff_up-option_value)/dS_up-(option_value-payoff_down)/dS_down)/(((self.S0+S_up)/2)-((self.S0+S_down)/2))
        return option_value,delta,gamma

In [68]:
eu_call=BinomialLRWithGreeks(50,52,r=0.05,T=2,N=300,sigma=0.3)
results=eu_call.price()
print('EU call')
print('Price: %s\nDelta: %s\nGamma: %s' % results)

EU call
Price: 18.28542226834646
Delta: 0.6978850971357399
Gamma: 0.0077717215740361


In [69]:
eu_put=BinomialLRWithGreeks(50,52,r=0.05,T=2,N=300,sigma=0.3,is_put=True)
results=eu_put.price()
print('EU put')
print('Price: %s\nDelta: %s\nGamma: %s' % results)

EU put
Price: 15.336968006216377
Delta: -0.3021149028642461
Gamma: 0.007771721574036346
